在视觉空间进行预测

In [1]:
import sys
sys.path.append('/mnt/workspace')

In [2]:
from py_file.M_attri import Att
from py_file.Get_Data import DATA
from py_file.data_set import MyDataSet
from torch.utils.data import DataLoader,Dataset,random_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import numpy as np
import matplotlib.pyplot as plt
import os
import math

In [3]:
# 定义训练的设备
if torch.cuda.is_available():
    device = torch.device('cuda:0') # 只有一张显卡的话，'cuda'和'cuda:0'是一样的
else:
    device = torch.device('cpu')

print(f'使用的设备为：{device}')

使用的设备为：cuda:0


In [4]:
trans = transforms.Resize(224)  # ResNet模型适合的图片大小为224x244
# 输入的张量需要带着批次维度和通道维度

# 准备数据以及模型

In [5]:
# 读取模型以及语义属性
import pickle
model = torch.load('../model_saved_pseudo/train_all.pth')
# model = model.to(device)
# model.eval()

model_fea = torch.nn.Sequential(*list(model.children())[:-2])  # 由于加了sigmoid层，所以这里是[:-2]
model_fea = model_fea.to(device)
model_fea.eval()

'''
attri = Att()
attri.compute_mul_defect_att()

single_defect_att = attri.single_defect_att
two_defect_att = attri.two_defect_att
three_defect_att = attri.three_defect_att
four_defect_att = attri.four_defect_att
mul_defect_att = attri.mul_defect_att
total_defect_att = attri.total_defect_att
'''

with open('../updated_semantic_all/updated_single_dict.pkl', 'rb') as file:
    single_defect_att = pickle.load(file)
with open('../updated_semantic_all/updated_mul_dict.pkl', 'rb') as file:
    mul_defect_att = pickle.load(file)
with open('../updated_semantic_all/updated_total_dict.pkl', 'rb') as file:
    total_defect_att = pickle.load(file)

from collections import Counter
# 加载二、三、四缺陷的字典
two_defect_att = {}
three_defect_att = {}
four_defect_att = {}
for label in mul_defect_att.keys():
    count = Counter(label)
    if count['+'] == 1:
        two_defect_att[label] = mul_defect_att[label]
    if count['+'] == 2:
        three_defect_att[label] = mul_defect_att[label]
    if count['+'] == 3:
        four_defect_att[label] = mul_defect_att[label]

In [6]:
test_data_path = '/mnt/workspace/DATA/test_WM.npz'
test_data = np.load(test_data_path)

att_dimen = len(single_defect_att['Center'])
print('att_dimen:', att_dimen)

att_dimen: 20


In [7]:
test_label = test_data['label_name']
test_wm = test_data['denoise_wm']
test_wm_tensor = trans(torch.reshape(torch.tensor(test_wm, dtype=torch.float32),(len(test_wm),1,52,52)))
print(test_wm_tensor.shape, test_label.shape)

torch.Size([7405, 1, 224, 224]) (7405,)


In [8]:
test_label_oh = test_data['label_one_hot']
# 通过one_hot标签给数据分为单缺陷、双缺陷、三缺陷以及四缺陷

test_single_wm = []  # 先定义列表，然后转换为tensor
test_single_label = []

test_two_wm = []
test_two_label = []

test_three_wm = []
test_three_label = []

test_four_wm = []
test_four_label = []

for i in range(len(test_label_oh)):
    if test_label_oh[i].sum() <= 1:
        test_single_wm.append(np.array(test_wm_tensor[i]))
        test_single_label.append(test_label[i])

    elif test_label_oh[i].sum() == 2:
        test_two_wm.append(np.array(test_wm_tensor[i]))
        test_two_label.append(test_label[i])

    elif test_label_oh[i].sum() == 3:
        test_three_wm.append(np.array(test_wm_tensor[i]))
        test_three_label.append(test_label[i])

    elif test_label_oh[i].sum() == 4:
        test_four_wm.append(np.array(test_wm_tensor[i]))
        test_four_label.append(test_label[i])


del test_data

In [9]:
test_size = len(test_wm_tensor)

# 定义需要的函数

In [10]:
def euclidean_distance(v1, v2):  # 参数v1,v2是np.array
    # 计算两个向量之间的欧氏距离
    distance = np.sqrt(np.sum((v1 - v2) ** 2))
    return distance

In [11]:
def predict_label(fea, vc):  # fea是numpy.ndarray类型
    min_dis = math.inf
    for c in vc.keys():
        dis = euclidean_distance(fea, vc[c])
        if dis < min_dis:
            min_dis = dis
            label = c
    return label

# 开始测试

## 加载视觉中心

In [12]:
with open('single_vc.pkl', 'rb') as file:
    single_vc = pickle.load(file)
with open('two_vc.pkl', 'rb') as file:
    two_vc = pickle.load(file)
with open('three_vc.pkl', 'rb') as file:
    three_vc = pickle.load(file)
with open('four_vc.pkl', 'rb') as file:
    four_vc = pickle.load(file)
with open('total_vc.pkl', 'rb') as file:
    total_vc = pickle.load(file)
mul_vc = {**two_vc, **three_vc, **four_vc}

## 单故障

In [13]:
single_wm_tensor = torch.tensor(np.array(test_single_wm), dtype=torch.float32)
single_dataset = MyDataSet(single_wm_tensor,test_single_label)
single_loader = DataLoader(single_dataset, batch_size=32, shuffle=False)
del single_wm_tensor

In [14]:
x = []
y = np.array(test_single_label)
with torch.no_grad():
    for imgs,labels in single_loader:  
        imgs = imgs.to(device)
        outputs = model_fea(imgs)

        for out in outputs:
            x.append(out.flatten().cpu())  # 输出展开,out的维度大小变为512，没有flatten的话维度大小就是(512,1,1)

x = np.array(x)
print(x.shape, y.shape)

del single_dataset, single_loader

(1605, 512) (1605,)


In [15]:
pre_y = []
for i in range(x.shape[0]):
    pre_y.append(predict_label(x[i], single_vc))
pre_y = np.array(pre_y)
acc = (pre_y==y).sum()/y.shape[0]
print(f'狭义搜索空间的准确率为{acc}')

狭义搜索空间的准确率为0.9819314641744549


In [16]:
pre_y = []
for i in range(x.shape[0]):
    pre_y.append(predict_label(x[i], total_vc))
pre_y = np.array(pre_y)
acc = (pre_y==y).sum()/y.shape[0]
print(f'广义搜索空间的准确率为{acc}')

广义搜索空间的准确率为0.9813084112149533


## 二故障

In [17]:
two_wm_tensor = torch.tensor(np.array(test_two_wm), dtype=torch.float32)
two_dataset = MyDataSet(two_wm_tensor,test_two_label)
two_loader = DataLoader(two_dataset, batch_size=32, shuffle=False)
del two_wm_tensor

In [18]:
x = []
y = np.array(test_two_label)
with torch.no_grad():
    for imgs,labels in two_loader:  
        imgs = imgs.to(device)
        outputs = model_fea(imgs)

        for out in outputs:
            x.append(out.flatten().cpu())  # 输出展开,out的维度大小变为512，没有flatten的话维度大小就是(512,1,1)

x = np.array(x)
print(x.shape, y.shape)

del two_dataset, two_loader

(2600, 512) (2600,)


In [19]:
pre_y = []
for i in range(x.shape[0]):
    pre_y.append(predict_label(x[i], two_vc))
pre_y = np.array(pre_y)
acc = (pre_y==y).sum()/y.shape[0]
print(f'狭义搜索空间的准确率为{acc}')

狭义搜索空间的准确率为0.9142307692307692


In [20]:
pre_y = []
for i in range(x.shape[0]):
    pre_y.append(predict_label(x[i], total_vc))
pre_y = np.array(pre_y)
acc = (pre_y==y).sum()/y.shape[0]
print(f'广义搜索空间的准确率为{acc}')

广义搜索空间的准确率为0.796923076923077


## 三故障

In [21]:
three_wm_tensor = torch.tensor(np.array(test_three_wm), dtype=torch.float32)
three_dataset = MyDataSet(three_wm_tensor,test_three_label)
three_loader = DataLoader(three_dataset, batch_size=32, shuffle=False)
del three_wm_tensor

In [22]:
x = []
y = np.array(test_three_label)
with torch.no_grad():
    for imgs,labels in three_loader:  
        imgs = imgs.to(device)
        outputs = model_fea(imgs)

        for out in outputs:
            x.append(out.flatten().cpu())  # 输出展开,out的维度大小变为512，没有flatten的话维度大小就是(512,1,1)

x = np.array(x)
print(x.shape, y.shape)

del three_dataset, three_loader

(2400, 512) (2400,)


In [23]:
pre_y = []
for i in range(x.shape[0]):
    pre_y.append(predict_label(x[i], three_vc))
pre_y = np.array(pre_y)
acc = (pre_y==y).sum()/y.shape[0]
print(f'狭义搜索空间的准确率为{acc}')

狭义搜索空间的准确率为0.7616666666666667


In [24]:
pre_y = []
for i in range(x.shape[0]):
    pre_y.append(predict_label(x[i], total_vc))
pre_y = np.array(pre_y)
acc = (pre_y==y).sum()/y.shape[0]
print(f'广义搜索空间的准确率为{acc}')

广义搜索空间的准确率为0.5541666666666667


## 四故障

In [25]:
four_wm_tensor = torch.tensor(np.array(test_four_wm), dtype=torch.float32)
four_dataset = MyDataSet(four_wm_tensor,test_four_label)
four_loader = DataLoader(four_dataset, batch_size=32, shuffle=False)
del four_wm_tensor

In [26]:
x = []
y = np.array(test_four_label)
with torch.no_grad():
    for imgs,labels in four_loader:  
        imgs = imgs.to(device)
        outputs = model_fea(imgs)

        for out in outputs:
            x.append(out.flatten().cpu())  # 输出展开,out的维度大小变为512，没有flatten的话维度大小就是(512,1,1)

x = np.array(x)
print(x.shape, y.shape)

del four_dataset, four_loader

(800, 512) (800,)


In [27]:
pre_y = []
for i in range(x.shape[0]):
    pre_y.append(predict_label(x[i], four_vc))
pre_y = np.array(pre_y)
acc = (pre_y==y).sum()/y.shape[0]
print(f'狭义搜索空间的准确率为{acc}')

狭义搜索空间的准确率为0.64375


In [28]:
pre_y = []
for i in range(x.shape[0]):
    pre_y.append(predict_label(x[i], total_vc))
pre_y = np.array(pre_y)
acc = (pre_y==y).sum()/y.shape[0]
print(f'广义搜索空间的准确率为{acc}')

广义搜索空间的准确率为0.3575


## 混合故障

In [29]:
test_mul_wm = test_two_wm + test_three_wm + test_four_wm
test_mul_label = test_two_label + test_three_label + test_four_label

In [30]:
mul_wm_tensor = torch.tensor(np.array(test_mul_wm), dtype=torch.float32)
mul_dataset = MyDataSet(mul_wm_tensor,test_mul_label)
mul_loader = DataLoader(mul_dataset, batch_size=32, shuffle=False)
del mul_wm_tensor

In [31]:
x = []
y = np.array(test_mul_label)
with torch.no_grad():
    for imgs,labels in mul_loader:  
        imgs = imgs.to(device)
        outputs = model_fea(imgs)

        for out in outputs:
            x.append(out.flatten().cpu())  # 输出展开,out的维度大小变为512，没有flatten的话维度大小就是(512,1,1)

x = np.array(x)
print(x.shape, y.shape)

del mul_dataset, mul_loader

(5800, 512) (5800,)


In [32]:
pre_y = []
for i in range(x.shape[0]):
    pre_y.append(predict_label(x[i], mul_vc))
pre_y = np.array(pre_y)
acc = (pre_y==y).sum()/y.shape[0]
print(f'狭义搜索空间的准确率为{acc}')

狭义搜索空间的准确率为0.6670689655172414


In [33]:
pre_y = []
for i in range(x.shape[0]):
    pre_y.append(predict_label(x[i], total_vc))
pre_y = np.array(pre_y)
acc = (pre_y==y).sum()/y.shape[0]
print(f'广义搜索空间的准确率为{acc}')

广义搜索空间的准确率为0.6358620689655172


## 所有故障

In [34]:
total_dataset = MyDataSet(test_wm_tensor,test_label)
total_loader = DataLoader(total_dataset, batch_size=32, shuffle=False)
del test_wm_tensor

In [35]:
x = []
y = np.array(test_label)
with torch.no_grad():
    for imgs,labels in total_loader:  
        imgs = imgs.to(device)
        outputs = model_fea(imgs)

        for out in outputs:
            x.append(out.flatten().cpu())  # 输出展开,out的维度大小变为512，没有flatten的话维度大小就是(512,1,1)

x = np.array(x)
print(x.shape, y.shape)

del total_dataset, total_loader

(7405, 512) (7405,)


In [36]:
pre_y = []
for i in range(x.shape[0]):
    pre_y.append(predict_label(x[i], total_vc))
pre_y = np.array(pre_y)
acc = (pre_y==y).sum()/y.shape[0]
print(f'准确率为{acc}')

准确率为0.7107359891964888
